# Generation of Penrose Tilings of Type P2 and P3 by Deflation

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import itertools

## Settings

In [ ]:
TRIANGLE_TYPES = {1, 2} # type 1: for P2-tilings, type 2: for P3-tilings
SHAPES = {'A', 'O'} # 'A': acute equilateral triangle, 'O': obtuse equilateral triangle 
SIDES = {'F', 'B'} # 'F': frontside, 'B': backside

LINE_COLOR = 'black'
TRIANGLE_COLOR_PER_TYPE_SHAPE_SIDE = {
    1: {
        'A': {
            'F': (1, 0, 0, 1),
            'B': (1, 0, 0, 0.5),
        },
        'O': {
            'F': (1, 0.5, 0, 1),
            'B': (1, 0.5, 0, 0.5),
        }
    },
    2: {
        'A': {
            'F': (0, 0, 1, 1),
            'B': (0, 0, 1, 0.5),
        },
        'O': {
            'F': (0, 0.5, 1, 1),
            'B': (0, 0.5, 1, 0.5),
        }
    }
}

## Data structres and helpers

In [ ]:
def rotate(point, origin, angle_deg):
    '''rotates the complex number point around the complex number origin by angle_deg in degrees (counter-clockwise)'''
    angle_rad = np.deg2rad(angle_deg)
    return (point - origin) * np.exp(1j * angle_rad) + origin

In [ ]:
class RobinsonTriangle:
    '''containter for Robinson triangles'''
    def __init__(self, t_type, shape, side, A, B):
        if t_type not in TRIANGLE_TYPES:
            raise ValueError('t_type must be in {}, but is {}'.format(triangle_types, t_type))
        self.t_type = t_type
        if shape not in SHAPES:
            raise ValueError('shape must be in {}, but is {}'.format(shapes, shape))
        self.shape = shape
        self.side = side
        if side not in SIDES:
            raise ValueError('side must be in {}, but is {}'.format(sides, side))
        self.side = side
        if abs(B - A) == 0:
            raise ValueError('points {} and {} are identical, but must be different'.format(A, B))
        if not isinstance(A, complex):
            raise ValueError('A must be a complex number but is {}'.format(type(A)))
        if not isinstance(B, complex):
            raise ValueError('B must be a complex number but is {}'.format(type(B)))
        self.A = A # the point at the top of the equilateral triangle as a complex number
        self.B = B # the point after A (anti-clockwise) as a complex number
        rot_angle_deg = 36 if self.shape == 'A' else 108
        self.C = rotate(B, A, rot_angle_deg)
    def __repr__(self):
        return '[t_type={}, shape={}, side={}, A={}, B={}, C={}]'.format(self.t_type, self.shape, self.side, self.A, self.B, self.C)
    def draw(self, draw_base_side=True, fill=True):
        li = [[self.A.real, self.B.real], [self.A.imag, self.B.imag], [self.A.real, self.C.real], [self.A.imag, self.C.imag]]
        if draw_base_side:
            li += [[self.B.real, self.C.real], [self.B.imag, self.C.imag]]
        plt.plot(*li, color='black')
        if fill:
            plt.fill([self.A.real, self.B.real, self.C.real], [self.A.imag, self.B.imag, self.C.imag], color=TRIANGLE_COLOR_PER_TYPE_SHAPE_SIDE[self.t_type][self.shape][self.side])
    def deflate(self):
        if self.t_type == 1:
            return []
            # raise NotImplementedError('deflation for t_type 1 is not implemented yet')
        elif self.t_type == 2:
            if self.shape == 'A':
                if self.side == 'F':
                    D = rotate(self.C, self.B, 36)
                    return [
                        RobinsonTriangle(self.t_type, 'A', 'F', self.B, self.C),
                        RobinsonTriangle(self.t_type, 'O', 'B', D, self.A),
                    ] 
                elif self.side == 'B':
                    D = rotate(self.B, self.C, 360 - 36)
                    return [
                        RobinsonTriangle(self.t_type, 'O', 'F', D, self.C),
                        RobinsonTriangle(self.t_type, 'A', 'B', self.C, D),
                    ] 
            elif self.shape == 'O':
                if self.side == 'F':
                    P = rotate(self.A, self.B, 360 - 36)
                    Q = rotate(self.A, P, 36)
                    return [
                        RobinsonTriangle(self.t_type, 'O', 'B', Q, self.B),
                        RobinsonTriangle(self.t_type, 'A', 'F', P, self.A),
                        RobinsonTriangle(self.t_type, 'O', 'F', P, self.C),
                    ]
                elif self.side == 'B':
                    P = rotate(self.A, self.C, 36)
                    Q = rotate(self.A, P, 360 - 36)
                return [
                        RobinsonTriangle(self.t_type, 'O', 'F', Q, P),
                        RobinsonTriangle(self.t_type, 'A', 'B', P, Q),
                        RobinsonTriangle(self.t_type, 'O', 'B', P, self.A),
                    ]
            

## Colors and deflation of Robinson triangles

In [ ]:
def draw_rt_and_deflation(i, t_type, shape, side):
    plt.sca(axs[i,0])
    plt.title('type {}, shape {} and side {}:'.format(t_type, shape, side))
    plt.axis('equal')
    plt.axis('off')
    rt = RobinsonTriangle(t_type, shape, side, complex(0), complex(1))
    rt.draw()
    plt.sca(axs[i,1])
    plt.title('Deflation of type {}, shape {} and side {}:'.format(t_type, shape, side))
    plt.axis('equal')
    plt.axis('off')
    for t in rt.deflate():
        t.draw()

fig, axs = plt.subplots(8, 2, figsize=(10, 20))
        
for (i, (t_type, shape, side)) in enumerate(itertools.product(TRIANGLE_TYPES, SHAPES, SIDES)):
    draw_rt_and_deflation(i, t_type, shape, side)

plt.show()

## Some more helpers

In [ ]:
def deflate_triangles(rts):
    '''deflates all robinson triangles within rts and returns the list of all deflated robinson triangles'''
    res = []
    for rt in rts:
        res += rt.deflate()
    return res

In [ ]:
def deflate(first_rt, iter_depth):
    '''returns the list of the robinson triangles after iter_depth deflations of the starting robinson triangle first_rt'''
    res = [first_rt]
    for i in range(iter_depth):
        res = deflate_triangles(res)
    return res

In [ ]:
def get_box(rts):
    '''returns the bounding box of a list of robinson triangles'''
    x_min = min([min(rt.A.real, rt.B.real, rt.C.real) for rt in rts])
    x_max = max([max(rt.A.real, rt.B.real, rt.C.real) for rt in rts])
    y_min = min([min(rt.A.imag, rt.B.imag, rt.C.imag) for rt in rts])
    y_max = max([max(rt.A.imag, rt.B.imag, rt.C.imag) for rt in rts])
    return (x_min, y_min), (x_max, y_max)

In [ ]:
def draw_rts(rts, draw_base_side=False, fill=False, fig_width=50, svg_file_name=''):
    '''draws the robinson triangles within rts'''
    (x_min, y_min), (x_max, y_max) = get_box(rts)
    y_scale = (x_max - x_min) / (y_max - y_min)
    plt.figure(figsize=(fig_width, fig_width / y_scale))
    plt.axis('equal')
    plt.axis('off')
    for t in rts:
        t.draw(draw_base_side=draw_base_side, fill=fill)
    if svg_file_name:
        plt.savefig(svg_file_name)
    plt.show()

## Draw Penrose tilings

In [ ]:
def draw_penrose_tiling(t_type=1, shape='A', side='F', iter_depth=8, A=complex(0), B=complex(1), draw_base_side=False, fill=True, fig_width=50, svg_file_name=''):
    '''draws the penrose tiling after iter_depth deflations, starting with a robinson triangle of type t_type, shape shape, side side, point A and point B.'''
    rt = RobinsonTriangle(t_type, shape, side, A, B)
    draw_rts(deflate(rt, iter_depth), draw_base_side=draw_base_side, fill=fill, fig_width=fig_width, svg_file_name=svg_file_name)

In [ ]:
draw_penrose_tiling(2, 'A', 'F', 8)

In [ ]:
draw_penrose_tiling(2, 'O', 'B', 8, draw_base_side=True, fill=False)

In [ ]:
for (t_type, shape, side) in itertools.product(TRIANGLE_TYPES, SHAPES, SIDES):
    if t_type == 2:
        draw_penrose_tiling(t_type, shape, side, 8, fill=False, svg_file_name='penrose_tiling_{}_{}_{}.svg'.format(t_type, shape, side))

In [ ]:
for (t_type, shape, side) in itertools.product(TRIANGLE_TYPES, SHAPES, SIDES):
    if t_type == 2:
        draw_penrose_tiling(t_type, shape, side, 8, svg_file_name='penrose_tiling_colored_{}_{}_{}.svg'.format(t_type, shape, side))